# Why are Telco's Customers Churning? 
## A Root Cause Analysis

### Sean Oslin

## Project planning phase

### Project goal/research question

### Deliverables

1. Jupyter notebook where your work takes place

2. CSV file that predicts churn for each customer

3. Python script that prepares data such that it can be fed into your model

4. Google slide summarizing your model

5. README.md file that contains a link to your google slides presentation, and instructions for how to use your python script(s)


### Minimum viable product

### Null and alternative hypotheses

H<sub>0</sub>:  Telco's data cannot provide any discernable causes for client churn

H<sub>a</sub>:  Reasons for client churn can be elucidated from Telco's data

### Python libraries used for analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from sklearn.linear_model import LinearRegression
from math import sqrt
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import warnings
warnings.filterwarnings("ignore")
from scipy import stats


### Python programming for this analysis imported from a text editor

In [2]:
import env
import wrangle_telco

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 27 columns):
customer_id                 7032 non-null object
gender                      7032 non-null object
senior_citizen              7032 non-null int64
partner                     7032 non-null object
dependents                  7032 non-null object
tenure                      7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
internet_service_type_id    7032 non-null int64
online_security             7032 non-null object
online_backup               7032 non-null object
device_protection           7032 non-null object
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
contract_type_id            7032 non-null int64
paperless_billing           7032 non-null object
payment_type_id             7032 non-null int64
monthly_charges 

### Import prepared Telco data from the text editor to this notebook

In [3]:
df_t = wrangle_telco.wrangle_telco()

In [4]:
pd.set_option('display.max_columns', None) #prevents 'head' from truncated the number of columns

In [5]:
df_t.head(5)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,internet_service_type_id,internet_service_type,payment_type_id,payment_type
0,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,No,No,No,No,No,1,Yes,1,48.20,340.35,No,1,Month-to-month,1,DSL,1,Electronic check
1,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,No,No,No,No,No,1,Yes,1,25.10,25.1,Yes,1,Month-to-month,1,DSL,1,Electronic check
2,0067-DKWBL,Male,1,No,No,2,Yes,No,1,Yes,No,No,No,No,No,1,Yes,1,49.25,91.1,Yes,1,Month-to-month,1,DSL,1,Electronic check
3,0096-BXERS,Female,0,Yes,No,6,Yes,Yes,1,No,No,No,No,No,No,1,No,1,50.35,314.55,No,1,Month-to-month,1,DSL,1,Electronic check
4,0156-FVPTA,Male,0,Yes,No,22,Yes,No,1,Yes,No,No,Yes,No,No,1,Yes,1,54.20,1152.7,Yes,1,Month-to-month,1,DSL,1,Electronic check


## First glance at data to assess data structure changes needed before analysis

In [6]:
wrangle_telco.peekatdata(df_t)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 27 columns):
customer_id                 7032 non-null object
gender                      7032 non-null object
senior_citizen              7032 non-null int64
partner                     7032 non-null object
dependents                  7032 non-null object
tenure                      7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
internet_service_type_id    7032 non-null int64
online_security             7032 non-null object
online_backup               7032 non-null object
device_protection           7032 non-null object
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
contract_type_id            7032 non-null int64
paperless_billing           7032 non-null object
payment_type_id             7032 non-null int64
monthly_charges 

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,internet_service_type_id,internet_service_type,payment_type_id,payment_type
0,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,No,No,No,No,No,1,Yes,1,48.20,340.35,No,1,Month-to-month,1,DSL,1,Electronic check
1,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,No,No,No,No,No,1,Yes,1,25.10,25.1,Yes,1,Month-to-month,1,DSL,1,Electronic check
2,0067-DKWBL,Male,1,No,No,2,Yes,No,1,Yes,No,No,No,No,No,1,Yes,1,49.25,91.1,Yes,1,Month-to-month,1,DSL,1,Electronic check
3,0096-BXERS,Female,0,Yes,No,6,Yes,Yes,1,No,No,No,No,No,No,1,No,1,50.35,314.55,No,1,Month-to-month,1,DSL,1,Electronic check
4,0156-FVPTA,Male,0,Yes,No,22,Yes,No,1,Yes,No,No,Yes,No,No,1,Yes,1,54.20,1152.7,Yes,1,Month-to-month,1,DSL,1,Electronic check
5,0191-ZHSKZ,Male,1,No,No,30,Yes,No,1,Yes,Yes,No,No,Yes,Yes,1,Yes,1,74.75,2111.3,No,1,Month-to-month,1,DSL,1,Electronic check
6,0212-ISBBF,Female,0,No,No,22,No,No phone service,1,No,Yes,No,No,Yes,Yes,1,Yes,1,50.35,1098.85,No,1,Month-to-month,1,DSL,1,Electronic check
7,0224-HJAPT,Male,0,No,No,5,Yes,Yes,1,No,No,No,No,No,No,1,No,1,47.15,223.15,Yes,1,Month-to-month,1,DSL,1,Electronic check
8,0224-RLWWD,Female,1,No,No,2,Yes,No,1,No,No,No,No,No,No,1,Yes,1,44.30,89.3,No,1,Month-to-month,1,DSL,1,Electronic check
9,0228-MAUWC,Male,0,No,No,19,Yes,Yes,1,No,No,Yes,Yes,No,No,1,No,1,59.55,1144.6,No,1,Month-to-month,1,DSL,1,Electronic check


### Remove duplicate columns

In [7]:
df_t = df_t.loc[:,~df_t.columns.duplicated()]

In [8]:
cols = df_t.columns.tolist() #3 columns used for data join have been eliminated
print(cols, end = " ")

['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service', 'multiple_lines', 'internet_service_type_id', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract_type_id', 'paperless_billing', 'payment_type_id', 'monthly_charges', 'total_charges', 'churn', 'contract_type', 'internet_service_type', 'payment_type'] 

### Check for duplicate customer ID numbers

In [9]:
duplicateRowsDF = df_t[df_t.duplicated(['customer_id'])]
duplicateRowsDF.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type


### Reorder columns so ID number preceeds the description

In [10]:
df_t = df_t[['customer_id',
 'gender','senior_citizen','partner','dependents','tenure','phone_service','multiple_lines','internet_service_type_id',\
             'internet_service_type','online_security','online_backup','device_protection','tech_support',\
             'streaming_tv','streaming_movies','contract_type_id','contract_type', 'paperless_billing',\
             'payment_type_id','payment_type','monthly_charges','total_charges','churn']]

### Convert yes/no and female/male variables to binary (0,1)

In [11]:
df_t.replace(to_replace=['No', 'Yes'], value=[0, 1], inplace = True)

In [12]:
df_t.replace(to_replace=['No phone service'], value=[0], inplace = True) #convert 'No phone service to 0'

In [13]:
df_t.replace(to_replace=['Female', 'Male'], value=[0, 1], inplace = True)

In [14]:
df_t.replace(to_replace=['No internet service'], value=[0], inplace = True)

In [15]:
df_t.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,internet_service_type,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,contract_type,paperless_billing,payment_type_id,payment_type,monthly_charges,total_charges,churn
0,0015-UOCOJ,0,1,0,0,7,1,0,1,DSL,1,0,0,0,0,0,1,Month-to-month,1,1,Electronic check,48.20,340.35,0
1,0023-HGHWL,1,1,0,0,1,0,0,1,DSL,0,0,0,0,0,0,1,Month-to-month,1,1,Electronic check,25.10,25.1,1
2,0067-DKWBL,1,1,0,0,2,1,0,1,DSL,1,0,0,0,0,0,1,Month-to-month,1,1,Electronic check,49.25,91.1,1
3,0096-BXERS,0,0,1,0,6,1,1,1,DSL,0,0,0,0,0,0,1,Month-to-month,0,1,Electronic check,50.35,314.55,0
4,0156-FVPTA,1,0,1,0,22,1,0,1,DSL,1,0,0,1,0,0,1,Month-to-month,1,1,Electronic check,54.20,1152.7,1


### Remove null values - all null values removed in MySQL

### Data types adjusted to best suit each variable 

In [19]:
df_t['total_charges'] = pd.to_numeric(df_t['total_charges'], errors='coerce')

In [20]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 24 columns):
customer_id                 7032 non-null object
gender                      7032 non-null int64
senior_citizen              7032 non-null int64
partner                     7032 non-null int64
dependents                  7032 non-null int64
tenure                      7032 non-null int64
phone_service               7032 non-null int64
multiple_lines              7032 non-null int64
internet_service_type_id    7032 non-null int64
internet_service_type       7032 non-null object
online_security             7032 non-null int64
online_backup               7032 non-null int64
device_protection           7032 non-null int64
tech_support                7032 non-null int64
streaming_tv                7032 non-null int64
streaming_movies            7032 non-null int64
contract_type_id            7032 non-null int64
contract_type               7032 non-null object
paperless_billing         